In [ ]:
!pip install -q transformers datasets peft accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import prepare_model_for_kbit_training
import torch

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # evita erro em padding

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",

    torch_dtype=torch.float16,
    quantization_config={
        "load_in_4bit": True,
        "bnb_4bit_compute_dtype": torch.float16,
        "bnb_4bit_use_double_quant": True,
        "bnb_4bit_quant_type": "nf4",
    },
)

model = prepare_model_for_kbit_training(model)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# !pip install --upgrade fsspec datasets


In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset("imdb", split="train[:1%]")
print(raw_dataset[0])


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [ ]:
# 1. Instalar pacotes (roda no Colab)
# !pip install -q transformers datasets peft accelerate bitsandbytes

# 2. Importar libs
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

# 3. Carregar dataset
raw_dataset = load_dataset("imdb", split="train[:80%]")

positives = raw_dataset.filter(lambda x: x["label"] == 1).select(range(500))
negatives = raw_dataset.filter(lambda x: x["label"] == 0).select(range(500))
from datasets import concatenate_datasets
raw_dataset = concatenate_datasets([positives, negatives]).shuffle(seed=42)


# 4. Carregar tokenizer e modelo TinyLlama quantizado em 4bit
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",

    torch_dtype=torch.float16,
    quantization_config={
        "load_in_4bit": True,
        "bnb_4bit_compute_dtype": torch.float16,
        "bnb_4bit_use_double_quant": True,
        "bnb_4bit_quant_type": "nf4",
    },
)

model = prepare_model_for_kbit_training(model)

# 5. Preprocessar dataset para treino causal LM
def preprocess_function(example):
    text = example["text"]
    label = example["label"]
    label_str = "positive" if label == 1 else "negative"
    prompt = f"Review: {text}\nSentiment:"
    completion = f" {label_str}"

    # Tokeniza prompt e completion juntos (entrada + saída)
    full_text = prompt + completion
    encodings = tokenizer(full_text, truncation=True, max_length=512, padding="max_length")

    # labels = input_ids para causal LM
    encodings["labels"] = encodings["input_ids"].copy()
    return encodings

train_dataset = raw_dataset.map(preprocess_function, remove_columns=raw_dataset.column_names)

# 6. Configurar LoRA para fine tuning
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # ajuste caso use outro modelo
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# 7. Argumentos do treinamento
training_args = TrainingArguments(
    output_dir="./qlora-imdb",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=50,
    save_total_limit=2,
    fp16=True,
    report_to="none",
    max_steps = 20,
)

# 8. Data collator para causal LM (não mask language modeling)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 9. Criar Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

# 10. Treinar
trainer.train()

# 11. Testar geração simples
model.eval()
prompt = "Review: I absolutely loved this movie. It was fantastic!\nSentiment:"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=10)

print(tokenizer.decode(output[0], skip_special_tokens=True))


Filter:   0%|          | 0/20000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.859800
20,2.769300


Review: I absolutely loved this movie. It was fantastic!
Sentiment: 10/10
Overall:


In [ ]:
example = raw_dataset[29]  # primeira linha
print("Text:", example["text"])
print("Label:", example["label"])


Text: I cannot stay indifferent to Lars van Trier's films. I consider 'Breaking the Waves' nothing less than a masterpiece. I loved 'Dancer in the Night'. I admired the idea in 'Dogville' but the overall exercise looked to me too dry and too theatrical, less cinema. 'Europa' which I see only now was a famous film at its time, succeeded in the US the relative success of an European film and got the Oscar for the best foreign language movie, but did not survive well the time in my opinion. It is also a too much explicit and extrovert exercise in cinema art to my taste.<br /><br />The story has a level of ambiguity that cannot escape the viewer. Treating the period that immediately followed the second world war not in the black and white colors of victors and vanquished, of executioners and victims but as rather ambiguous times when people of both sides were fighting for survival in the aftermath of a catastrophic event that change the lives of nations and individuals forever is still a s

In [ ]:
# for i, example in enumerate(raw_dataset):
  #   print(f"Example {i}:")
  #  print("Text:", example["text"])
  #  print("Label:", example["label"])
  #  print("---------")


In [ ]:
negatives

Dataset({
    features: ['text', 'label'],
    num_rows: 500
})

In [ ]:
print(f"Tamanho do dataset: {len(raw_dataset)}")

labels = [ex["label"] for ex in raw_dataset]
print(f"Labels únicos: {set(labels)}")
print(f"Quantidade por label:")
print({label: labels.count(label) for label in set(labels)})


Tamanho do dataset: 1000
Labels únicos: {0, 1}
Quantidade por label:
{0: 500, 1: 500}


In [ ]:
Classify the sentiment of the following movie review as either "positive" or "negative".

Review: I really do like so much this movie
Sentiment:

In [ ]:
# 11. Testar geração simples
model.eval()
prompt = "Review: I absolutely loved this movie. It was fantastic!\nSentiment:"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=10)

print(tokenizer.decode(output[0], skip_special_tokens=True))


Review: I absolutely loved this movie. It was fantastic!
Sentiment: 10/10
Overall:


In [ ]:
model.eval()
prompt = "Review: I absolutely loved this movie. It was fantastic!\nSentiment:"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=3,                  # limita a resposta
        eos_token_id=tokenizer.eos_token_id,
        do_sample=False                    # geração determinística
    )

text_out = tokenizer.decode(output[0], skip_special_tokens=True)

# Pega só o que vem depois de "Sentiment:"
sentiment = text_out.split("Sentiment:")[-1].strip().split()[0]

print("Sentiment:", sentiment)


Sentiment: 10


In [ ]:
import torch

# Seu modelo e tokenizer já devem estar carregados acima:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# model = AutoModelForCausalLM.from_pretrained("seu-modelo")
# tokenizer = AutoTokenizer.from_pretrained("seu-modelo")
# model.to("cuda" if torch.cuda.is_available() else "cpu")

model.eval()

def classify_sentiment(review_text):
    prompt = (
        f"Review: {review_text}\n"
        "Sentiment (0=negative, 1=positive):"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=3,  # suficiente para " 0" ou " 1"
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
        )

    text_out = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extrai o número após "Sentiment:"
    try:
        prediction = text_out.split("Sentiment (0=negative, 1=positive):")[-1].strip().split()[0]
        if prediction == "1":
            return "positive"
        elif prediction == "0":
            return "negative"
        else:
            return "unknown"
    except:
        return "unknown"

# Testes com 3 frases
reviews = [
    "I absolutely loved this movie. It was fantastic!",
    "I really did not enjoy this film at all.",
    "It was okay, not great but not terrible.",
]

for rev in reviews:
    print(f"Review: {rev}")
    print("Predicted sentiment:", classify_sentiment(rev))
    print("-" * 40)


Review: I absolutely loved this movie. It was fantastic!
Predicted sentiment: unknown
----------------------------------------
Review: I really did not enjoy this film at all.
Predicted sentiment: negative
----------------------------------------
Review: It was okay, not great but not terrible.
Predicted sentiment: negative
----------------------------------------


In [ ]:
print(classify_sentiment("This movie was terrible and boring."))
print(classify_sentiment("Absolutely fantastic and enjoyable film!"))
print(classify_sentiment("I don't know what to think about this movie."))


positive
positive
positive


In [ ]:
raw_dataset = load_dataset("imdb", split="train[:20%]")


In [ ]:
raw_dataset.select(range(5))

Dataset({
    features: ['text', 'label'],
    num_rows: 5
})

In [ ]:
import torch

# Certifique-se de já ter o modelo e tokenizer carregados:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# model = AutoModelForCausalLM.from_pretrained("seu-modelo")
# tokenizer = AutoTokenizer.from_pretrained("seu-modelo")
# model.to("cuda" if torch.cuda.is_available() else "cpu")

model.eval()

def classify_sentiment(review_text):
    # Prompt compatível com o treinamento
    prompt = (
        f"Review: {review_text}\n"
        "Sentiment (0=negative, 1=positive):"
    )

    # Tokeniza e envia para o mesmo dispositivo do modelo
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=3,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
        )

    # Decodifica saída do modelo
    text_out = tokenizer.decode(output[0], skip_special_tokens=True)

    # Exibe saída bruta para debug
    print("DEBUG >> Geração completa:\n", text_out)

    try:
        # Extrai a predição depois de "Sentiment:"
        prediction = text_out.split("Sentiment (0=negative, 1=positive):")[-1].strip().split()[0]

        # Mapeia predição para label
        if prediction in ["1", "10", "1/1", "10/10", "positive"]:
            return "positive"
        elif prediction in ["0", "0/1", "0/10", "negative"]:
            return "negative"
        else:
            return "unknown"
    except:
        return "unknown"

# Testes com várias frases
reviews = [
    "I absolutely loved this movie. It was fantastic!",
    "I really did not enjoy this film at all.",
    "It was okay, not great but not terrible.",
]

for rev in reviews:
    print(f"\nReview: {rev}")
    print("Predicted sentiment:", classify_sentiment(rev))
    print("-" * 40)



Review: I absolutely loved this movie. It was fantastic!
DEBUG >> Geração completa:
 Review: I absolutely loved this movie. It was fantastic!
Sentiment (0=negative, 1=positive): 5/
Predicted sentiment: unknown
----------------------------------------

Review: I really did not enjoy this film at all.
DEBUG >> Geração completa:
 Review: I really did not enjoy this film at all.
Sentiment (0=negative, 1=positive): 0

Predicted sentiment: negative
----------------------------------------

Review: It was okay, not great but not terrible.
DEBUG >> Geração completa:
 Review: It was okay, not great but not terrible.
Sentiment (0=negative, 1=positive): 0

Predicted sentiment: negative
----------------------------------------


In [ ]:
import torch

model.eval()

def classify_sentiment(review_text):
    # Prompt compatível com o modelo treinado
    prompt = (
        f"Review: {review_text}\n"
        "Sentiment (0=negative, 1=positive):"
    )

    # Prepara entrada
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Gera saída
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=3,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
        )

    # Decodifica a saída do modelo
    text_out = tokenizer.decode(output[0], skip_special_tokens=True)
    print("DEBUG >> Geração completa:\n", text_out)

    try:
        # Extrai o valor gerado após "Sentiment:"
        prediction = text_out.split("Sentiment (0=negative, 1=positive):")[-1].strip().split()[0]

        # Interpreta a saída do modelo
        if any(prediction.startswith(p) for p in ["1", "5", "10"]):
            return "positive"
        elif prediction.startswith("0"):
            return "negative"
        else:
            return "unknown"
    except:
        return "unknown"

# Lista de reviews para teste
reviews = [
    "I liked this movie. It was very good!",
    "I really did not enjoy this film at all.",
    "It was okay, not great but not terrible.",
]

# Aplica classificação em cada review
for rev in reviews:
    print(f"\nReview: {rev}")
    print("Predicted sentiment:", classify_sentiment(rev))
    print("-" * 40)



Review: I liked this movie. It was very good!
DEBUG >> Geração completa:
 Review: I liked this movie. It was very good!
Sentiment (0=negative, 1=positive): 1.
Predicted sentiment: positive
----------------------------------------

Review: I really did not enjoy this film at all.
DEBUG >> Geração completa:
 Review: I really did not enjoy this film at all.
Sentiment (0=negative, 1=positive): 0

Predicted sentiment: negative
----------------------------------------

Review: It was okay, not great but not terrible.
DEBUG >> Geração completa:
 Review: It was okay, not great but not terrible.
Sentiment (0=negative, 1=positive): 0

Predicted sentiment: negative
----------------------------------------
